In [196]:
# importing all the necessary libraries
import pandas as pd
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle

In [ ]:
def data_prep(df):
    df = df.drop(columns=["INDICATOR","FLAG"])

In [26]:
# importing the dataset
df = pd.read_csv('Drug_overdose_death_rates__by_drug_type__sex__age__race__and_Hispanic_origin__United_States_20240518.csv')
df.head()

,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,FLAG
0,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,1999,1,All ages,1.1,6.1,NaN
1,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2000,2,All ages,1.1,6.2,NaN
2,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2001,3,All ages,1.1,6.8,NaN
3,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2002,4,All ages,1.1,8.2,NaN
4,Drug overdose death rates,All drug overdose deaths,0,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.1,2003,5,All ages,1.1,8.9,NaN


In [27]:
df.describe()

,PANEL_NUM,UNIT_NUM,STUB_NAME_NUM,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE_NUM,ESTIMATE
count,6228.000000,6228.000000,6228.000000,6228.000000,6228.000000,6228.000000,6228.000000,5117.000000
mean,2.500000,1.578035,3.028902,3.383006,2008.664740,10.664740,1.354913,4.743443
std,1.707962,0.493913,1.447036,1.526819,5.849512,5.849512,0.301459,6.424471
min,0.000000,1.000000,0.000000,0.100000,1999.000000,1.000000,1.100000,0.000000
25%,1.000000,1.000000,2.000000,2.100000,2004.000000,6.000000,1.100000,0.800000
50%,2.500000,2.000000,3.000000,3.220000,2009.000000,11.000000,1.200000,2.100000
75%,4.000000,2.000000,4.000000,4.600000,2014.000000,16.000000,1.600000,6.000000
max,5.000000,2.000000,5.000000,5.930000,2018.000000,20.000000,1.910000,54.300000


In [29]:

# check for any null values

In [30]:
#Print a list of categorical variables
categorical_variables = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Variables:", categorical_variables)

Categorical Variables: ['PANEL', 'UNIT', 'STUB_NAME', 'STUB_LABEL', 'AGE']


In [145]:
numerical_variables = df.select_dtypes(include=['number']).columns.tolist()
print("Numerical Variables:", numerical_variables)

Numerical Variables: ['YEAR', 'ESTIMATE', 'WHITE', 'NATIVE_BLACK', 'BLACK_AMERICAN', 'AMERICAN_INDIAN', 'ASIAN_PACIFIC_ISLANDER', 'ASIAN', 'NATIVE_HAWAIIAN', 'HISPANIC_LATINO_FLAG', 'HISPANIC_LATINO_RACE']


In [32]:
# Get value counts for each categorical column
value_counts_dict = {}
for col in categorical_variables:
    value_counts_dict[col] = df[col].value_counts()

# Print the value counts for each categorical column
for col, counts in value_counts_dict.items():
    print(f"Value counts for column {col}:\n{counts}\n")

Value counts for column PANEL:
PANEL
All drug overdose deaths                                                         1038
Drug overdose deaths involving any opioid                                        1038
Drug overdose deaths involving natural and semisynthetic opioids                 1038
Drug overdose deaths involving methadone                                         1038
Drug overdose deaths involving other synthetic opioids (other than methadone)    1038
Drug overdose deaths involving heroin                                            1038
Name: count, dtype: int64

Value counts for column UNIT:
UNIT
Deaths per 100,000 resident population, crude           3600
Deaths per 100,000 resident population, age-adjusted    2628
Name: count, dtype: int64

Value counts for column STUB_NAME:
STUB_NAME
Sex and age                                       2160
Sex and race and Hispanic origin                  1200
Age                                               1080
Sex and race              

In [33]:
df.isnull().sum()

PANEL                0
PANEL_NUM            0
UNIT                 0
UNIT_NUM             0
STUB_NAME            0
STUB_NAME_NUM        0
STUB_LABEL           0
STUB_LABEL_NUM       0
YEAR                 0
YEAR_NUM             0
AGE                  0
AGE_NUM              0
ESTIMATE          1111
dtype: int64

In [34]:
df = df.dropna()

In [35]:
df.shape

(5117, 13)

In [36]:
df = df.drop(columns=["PANEL_NUM","UNIT_NUM","STUB_NAME_NUM","STUB_LABEL_NUM","YEAR_NUM","AGE_NUM"])

In [37]:
df.head()

,PANEL,UNIT,STUB_NAME,STUB_LABEL,YEAR,AGE,ESTIMATE
0,All drug overdose deaths,"Deaths per 100,000 resident population, age-ad...",Total,All persons,1999,All ages,6.1
1,All drug overdose deaths,"Deaths per 100,000 resident population, age-ad...",Total,All persons,2000,All ages,6.2
2,All drug overdose deaths,"Deaths per 100,000 resident population, age-ad...",Total,All persons,2001,All ages,6.8
3,All drug overdose deaths,"Deaths per 100,000 resident population, age-ad...",Total,All persons,2002,All ages,8.2
4,All drug overdose deaths,"Deaths per 100,000 resident population, age-ad...",Total,All persons,2003,All ages,8.9


In [102]:
# Create the Gender column
df['GENDER'] = df['STUB_LABEL'].apply(lambda x: 'Male' if 'Male' in x else 'Female' if 'Female' in x else 'Unknown')

# Creating are you hispanic latino column 
df['HISPANIC_LATINO_FLAG'] = df['STUB_LABEL'].apply(
        lambda x: 0 if  'Female: Not Hispanic or Latino: American Indian or Alaska Native' in x
        or 'Female: Not Hispanic or Latino: Black' in x
        or 'Female: Not Hispanic or Latino: White' in x
        or 'Male: Not Hispanic or Latino: American Indian or Alaska Native' in x
        or 'Male: Not Hispanic or Latino: Black' in x
        or 'Male: Not Hispanic or Latino: White' in x
        or 'Male: Not Hispanic or Latino: Asian or Pacific Islander' in x
        or 'Female: Not Hispanic or Latino: Asian or Pacific Islander' in x
        or 'Male: Not Hispanic or Latino: Asian' in x
        or 'Male: Not Hispanic or Latino: Native Hawaiian or Other Pacific Islander' in x
        or 'Female: Not Hispanic or Latino: Asian' in x
        or 'Female: Not Hispanic or Latino: Native Hawaiian or Other Pacific Islander' in x        
        else 1)

df['HISPANIC_LATINO_RACE'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Male: Hispanic or Latino: All races' in x or 'Female: Hispanic or Latino: All races' in x else 0)

df['WHITE'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Female: White' in x 
                                     or 'Male: White' in x 
                                     or 'Female: Not Hispanic or Latino: White' in x 
                                     or 'Male: Not Hispanic or Latino: White' in x
                                     else 0)

df['NATIVE_BLACK'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Female: Not Hispanic or Latino: Black' in x 
                                     or 'Male: Not Hispanic or Latino: Black' in x
                                     else 0)

df['BLACK_AMERICAN'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Female: Black or African American' in x 
                                     or 'Male: Black or African American' in x
                                     else 0)

df['AMERICAN_INDIAN'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Female: Not Hispanic or Latino: American Indian or Alaska Native' in x 
                                     or 'Male: Not Hispanic or Latino: American Indian or Alaska Native' in x
                                     or 'Male: American Indian or Alaska Native' in x
                                     or 'Female: American Indian or Alaska Native' in x
                                     else 0)

df['ASIAN_PACIFIC_ISLANDER'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Male: Asian or Pacific Islander' in x 
                                     or 'Female: Asian or Pacific Islander' in x
                                     or 'Male: Not Hispanic or Latino: Asian or Pacific Islander' in x
                                     or 'Female: Not Hispanic or Latino: Asian or Pacific Islander' in x
                                     else 0)

df['ASIAN'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Male: Not Hispanic or Latino: Asian' in x 
                                     or 'Female: Not Hispanic or Latino: Asian' in x
                                     else 0)

df['NATIVE_HAWAIIAN'] = df['STUB_LABEL'].apply(lambda x: 1 if 'Male: Not Hispanic or Latino: Native Hawaiian or Other Pacific Islander' in x 
                                     or 'Female: Not Hispanic or Latino: Native Hawaiian or Other Pacific Islander' in x
                                     else 0)


In [103]:
# Get value counts for the STUB_LABEL column
value_counts = df['GENDER'].value_counts()
print(value_counts)

GENDER
Male       2017
Female     1967
Unknown    1133
Name: count, dtype: int64


In [104]:
df.columns

Index(['PANEL', 'UNIT', 'STUB_NAME', 'STUB_LABEL', 'YEAR', 'AGE', 'ESTIMATE',
       'GENDER', 'WHITE', 'NATIVE_BLACK', 'BLACK_AMERICAN', 'AMERICAN_INDIAN',
       'ASIAN_PACIFIC_ISLANDER', 'ASIAN', 'NATIVE_HAWAIIAN',
       'HISPANIC_LATINO_FLAG', 'HISPANIC_LATINO_RACE'],
      dtype='object')

In [105]:
df.head()

,PANEL,UNIT,STUB_NAME,STUB_LABEL,YEAR,AGE,ESTIMATE,GENDER,WHITE,NATIVE_BLACK,BLACK_AMERICAN,AMERICAN_INDIAN,ASIAN_PACIFIC_ISLANDER,ASIAN,NATIVE_HAWAIIAN,HISPANIC_LATINO_FLAG,HISPANIC_LATINO_RACE
0,All,Age_Adjusted,Total,All persons,1999,All ages,6.1,Unknown,0,0,0,0,0,0,0,1,0
1,All,Age_Adjusted,Total,All persons,2000,All ages,6.2,Unknown,0,0,0,0,0,0,0,1,0
2,All,Age_Adjusted,Total,All persons,2001,All ages,6.8,Unknown,0,0,0,0,0,0,0,1,0
3,All,Age_Adjusted,Total,All persons,2002,All ages,8.2,Unknown,0,0,0,0,0,0,0,1,0
4,All,Age_Adjusted,Total,All persons,2003,All ages,8.9,Unknown,0,0,0,0,0,0,0,1,0


In [91]:
word_map = {
    "All drug overdose deaths": "All",
    "Drug overdose deaths involving any opioid": "Any_Opioid",
    "Drug overdose deaths involving natural and semisynthetic opioids": "Natural_Opioids",
    "Drug overdose deaths involving other synthetic opioids (other than methadone)": "Other_Synthetic",
    "Drug overdose deaths involving methadone": "Methadone",
    "Drug overdose deaths involving heroin": "Heroin"
}
# Replace values in the 'Panel' column with one-word representations
df["PANEL"] = df["PANEL"].replace(word_map)
df['PANEL'].value_counts()
word_map2 = {
    "Deaths per 100,000 resident population, crude": "Crude",
    "Deaths per 100,000 resident population, age-adjusted": "Age_Adjusted",
}
# Replace values in the 'Unit' column with one-word representations
df["UNIT"] = df["UNIT"].replace(word_map2)
df['UNIT'].value_counts()

UNIT
Crude           2904
Age_Adjusted    2213
Name: count, dtype: int64

In [106]:
df.head()

,PANEL,UNIT,STUB_NAME,STUB_LABEL,YEAR,AGE,ESTIMATE,GENDER,WHITE,NATIVE_BLACK,BLACK_AMERICAN,AMERICAN_INDIAN,ASIAN_PACIFIC_ISLANDER,ASIAN,NATIVE_HAWAIIAN,HISPANIC_LATINO_FLAG,HISPANIC_LATINO_RACE
0,All,Age_Adjusted,Total,All persons,1999,All ages,6.1,Unknown,0,0,0,0,0,0,0,1,0
1,All,Age_Adjusted,Total,All persons,2000,All ages,6.2,Unknown,0,0,0,0,0,0,0,1,0
2,All,Age_Adjusted,Total,All persons,2001,All ages,6.8,Unknown,0,0,0,0,0,0,0,1,0
3,All,Age_Adjusted,Total,All persons,2002,All ages,8.2,Unknown,0,0,0,0,0,0,0,1,0
4,All,Age_Adjusted,Total,All persons,2003,All ages,8.9,Unknown,0,0,0,0,0,0,0,1,0


In [ ]:
df = df.drop(columns="STUB_LABEL")

In [133]:
#Print a list of categorical variables
categorical_variables2 = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Variables:", categorical_variables2)

Categorical Variables: ['PANEL', 'UNIT', 'AGE', 'GENDER']


In [134]:
df.columns

Index(['PANEL', 'UNIT', 'YEAR', 'AGE', 'ESTIMATE', 'GENDER', 'WHITE',
       'NATIVE_BLACK', 'BLACK_AMERICAN', 'AMERICAN_INDIAN',
       'ASIAN_PACIFIC_ISLANDER', 'ASIAN', 'NATIVE_HAWAIIAN',
       'HISPANIC_LATINO_FLAG', 'HISPANIC_LATINO_RACE'],
      dtype='object')

In [155]:
# Perform one-hot encoding on 'STUB_LABEL' column
df_encoded = pd.get_dummies(df, columns=categorical_variables2)
df_encoded.columns

Index(['YEAR', 'ESTIMATE', 'WHITE', 'NATIVE_BLACK', 'BLACK_AMERICAN',
       'AMERICAN_INDIAN', 'ASIAN_PACIFIC_ISLANDER', 'ASIAN', 'NATIVE_HAWAIIAN',
       'HISPANIC_LATINO_FLAG', 'HISPANIC_LATINO_RACE', 'PANEL_All',
       'PANEL_Any_Opioid', 'PANEL_Heroin', 'PANEL_Methadone',
       'PANEL_Natural_Opioids', 'PANEL_Other_Synthetic', 'UNIT_Age_Adjusted',
       'UNIT_Crude', 'AGE_15-24 years', 'AGE_25-34 years', 'AGE_35-44 years',
       'AGE_45-54 years', 'AGE_55-64 years', 'AGE_65-74 years',
       'AGE_75-84 years', 'AGE_85 years and over', 'AGE_All ages',
       'AGE_Under 15 years', 'GENDER_Female', 'GENDER_Male', 'GENDER_Unknown'],
      dtype='object')

In [158]:
# Convert all boolean columns to integer columns with 1 and 0
bool_cols = df_encoded.select_dtypes(include='bool').columns
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)

In [ ]:
# Convert boolean values to integers
df_encoded = df_encoded.astype(int)
# Display the first few rows of the resulting DataFrame
print(df_encoded.head())

In [136]:
df_encoded.head()

,YEAR,ESTIMATE,WHITE,NATIVE_BLACK,BLACK_AMERICAN,AMERICAN_INDIAN,ASIAN_PACIFIC_ISLANDER,ASIAN,NATIVE_HAWAIIAN,HISPANIC_LATINO_FLAG,...,AGE_45-54 years,AGE_55-64 years,AGE_65-74 years,AGE_75-84 years,AGE_85 years and over,AGE_All ages,AGE_Under 15 years,GENDER_Female,GENDER_Male,GENDER_Unknown
0,1999,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,2000,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,2001,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
3,2002,8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,2003,8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [160]:
df_encoded.dtypes

YEAR                        int64
ESTIMATE                  float64
WHITE                       int64
NATIVE_BLACK                int64
BLACK_AMERICAN              int64
AMERICAN_INDIAN             int64
ASIAN_PACIFIC_ISLANDER      int64
ASIAN                       int64
NATIVE_HAWAIIAN             int64
HISPANIC_LATINO_FLAG        int64
HISPANIC_LATINO_RACE        int64
PANEL_All                   int32
PANEL_Any_Opioid            int32
PANEL_Heroin                int32
PANEL_Methadone             int32
PANEL_Natural_Opioids       int32
PANEL_Other_Synthetic       int32
UNIT_Age_Adjusted           int32
UNIT_Crude                  int32
AGE_15-24 years             int32
AGE_25-34 years             int32
AGE_35-44 years             int32
AGE_45-54 years             int32
AGE_55-64 years             int32
AGE_65-74 years             int32
AGE_75-84 years             int32
AGE_85 years and over       int32
AGE_All ages                int32
AGE_Under 15 years          int32
GENDER_Female 

In [161]:
# Change column names to lowercase and replace spaces with underscores
df_encoded.columns = df_encoded.columns.str.lower().str.replace(' ', '_')

# Display updated column names
print("Updated column names:", df_encoded.columns)

# Display the DataFrame to verify changes
print(df_encoded)

Updated column names: Index(['year', 'estimate', 'white', 'native_black', 'black_american',
       'american_indian', 'asian_pacific_islander', 'asian', 'native_hawaiian',
       'hispanic_latino_flag', 'hispanic_latino_race', 'panel_all',
       'panel_any_opioid', 'panel_heroin', 'panel_methadone',
       'panel_natural_opioids', 'panel_other_synthetic', 'unit_age_adjusted',
       'unit_crude', 'age_15-24_years', 'age_25-34_years', 'age_35-44_years',
       'age_45-54_years', 'age_55-64_years', 'age_65-74_years',
       'age_75-84_years', 'age_85_years_and_over', 'age_all_ages',
       'age_under_15_years', 'gender_female', 'gender_male', 'gender_unknown'],
      dtype='object')
      year  estimate  white  native_black  black_american  american_indian  \
0     1999       6.1      0             0               0                0   
1     2000       6.2      0             0               0                0   
2     2001       6.8      0             0               0                0 

In [162]:
df_encoded.head()

,year,estimate,white,native_black,black_american,american_indian,asian_pacific_islander,asian,native_hawaiian,hispanic_latino_flag,...,age_45-54_years,age_55-64_years,age_65-74_years,age_75-84_years,age_85_years_and_over,age_all_ages,age_under_15_years,gender_female,gender_male,gender_unknown
0,1999,6.1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,2000,6.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,2001,6.8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
3,2002,8.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,2003,8.9,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [142]:
df_encoded['estimate'] = df_encoded['estimate'].astype(float).round(2)

# Display updated DataFrame
print("\nUpdated DataFrame with all integer columns converted:")


Updated DataFrame with all integer columns converted:


In [163]:
df_encoded

,year,estimate,white,native_black,black_american,american_indian,asian_pacific_islander,asian,native_hawaiian,hispanic_latino_flag,...,age_45-54_years,age_55-64_years,age_65-74_years,age_75-84_years,age_85_years_and_over,age_all_ages,age_under_15_years,gender_female,gender_male,gender_unknown
0,1999,6.1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,2000,6.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,2001,6.8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
3,2002,8.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,2003,8.9,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6221,2018,5.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6222,2018,4.4,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6223,2018,3.4,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
6224,2018,2.3,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0


In [165]:
df_encoded.columns

Index(['year', 'estimate', 'white', 'native_black', 'black_american',
       'american_indian', 'asian_pacific_islander', 'asian', 'native_hawaiian',
       'hispanic_latino_flag', 'hispanic_latino_race', 'panel_all',
       'panel_any_opioid', 'panel_heroin', 'panel_methadone',
       'panel_natural_opioids', 'panel_other_synthetic', 'unit_age_adjusted',
       'unit_crude', 'age_15-24_years', 'age_25-34_years', 'age_35-44_years',
       'age_45-54_years', 'age_55-64_years', 'age_65-74_years',
       'age_75-84_years', 'age_85_years_and_over', 'age_all_ages',
       'age_under_15_years', 'gender_female', 'gender_male', 'gender_unknown'],
      dtype='object')

In [166]:
df_encoded = df_encoded.drop(columns=["panel_all","age_all_ages","gender_unknown"])

In [167]:
df_encoded.head()

,year,estimate,white,native_black,black_american,american_indian,asian_pacific_islander,asian,native_hawaiian,hispanic_latino_flag,...,age_25-34_years,age_35-44_years,age_45-54_years,age_55-64_years,age_65-74_years,age_75-84_years,age_85_years_and_over,age_under_15_years,gender_female,gender_male
0,1999,6.1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2000,6.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2001,6.8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2002,8.2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2003,8.9,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [168]:
df.to_csv('cleaned_dataset.csv')

In [180]:
df_encoded.shape

(5117, 29)

In [197]:
# Calculate VIF for each predictor variable
vif_data = pd.DataFrame()
vif_data["Variable"] = df_encoded.columns
vif_data["VIF"] = [variance_inflation_factor(df_encoded.values, i) for i in range(df_encoded.shape[1])]

print(vif_data)

                  Variable           VIF
0                     year      1.413293
1                 estimate      3.186248
2                    white      2.731578
3             native_black      2.746361
4           black_american      1.631711
5          american_indian      2.276052
6   asian_pacific_islander      2.473875
7                    asian      2.369171
8          native_hawaiian      1.009042
9     hispanic_latino_flag      3.170971
10    hispanic_latino_race      1.638510
11        panel_any_opioid      1.881987
12            panel_heroin      2.614890
13         panel_methadone      2.816926
14   panel_natural_opioids      2.518676
15   panel_other_synthetic      2.596931
16       unit_age_adjusted  73239.186443
17              unit_crude  96082.186292
18         age_15-24_years      1.861542
19         age_25-34_years      1.901301
20         age_35-44_years      1.935601
21         age_45-54_years      1.941785
22         age_55-64_years      1.838198
23         age_6

In [188]:
## having both the units (age adjusted and crude)

### Random Forest Regression

In [181]:
# Preprocess the data
X = df_encoded.drop('estimate', axis=1)  # Features
y = df_encoded['estimate']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'R-squared: {r2:.2f}')


Root Mean Squared Error: 1.34
R-squared: 0.96


### Gradient Boosting Regressor

In [185]:
# Create and train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 12.98
R-squared: 0.96


### Linear Regression

In [187]:
# Create and train the Linear Regression model
lin_regressor = LinearRegression()
lin_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lin_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 14.02
R-squared: 0.96


In [189]:
### having only one Deaths per 100,000 resident population unit

In [190]:
df_encoded.columns

Index(['year', 'estimate', 'white', 'native_black', 'black_american',
       'american_indian', 'asian_pacific_islander', 'asian', 'native_hawaiian',
       'hispanic_latino_flag', 'hispanic_latino_race', 'panel_any_opioid',
       'panel_heroin', 'panel_methadone', 'panel_natural_opioids',
       'panel_other_synthetic', 'unit_age_adjusted', 'unit_crude',
       'age_15-24_years', 'age_25-34_years', 'age_35-44_years',
       'age_45-54_years', 'age_55-64_years', 'age_65-74_years',
       'age_75-84_years', 'age_85_years_and_over', 'age_under_15_years',
       'gender_female', 'gender_male'],
      dtype='object')

In [ ]:
## dropping the unit column 
df_encoded_new = df.drop(columns=)